In [39]:
from z3 import Optimize, Real, Int, ToInt, And, Or, Implies, sat


def gift_tax_calculator(
    # period_choice 代表使用者選擇之贈與日:
    # (1) 民國98年1月23日至106年5月11日
    # (2) 民國106年5月12日至110年12月31日
    # (3) 民國111年1月1日(含)至民國113年12月31日
    # (4) 民國114年1月1日(含)以後
    period_choice: int,

    # ====== 以下對應「本次贈與總額 (1+2+3+4)」 ======
    land_value,
    ground_value,
    house_value,
    others_value,

    # ====== 四.不計入贈與總額之財產(1+2+3) ======
    not_included_land,
    not_included_house,
    not_included_others,

    # ====== (98 年舊制時需要使用者直接輸入的「本年度內剩餘尚未使用之免稅額」) ======
    remaining_exemption_98=0,

    # ====== (106/111/114 年，需同一年內先前贈與總額，用於合併計算) ======
    previous_gift_sum_in_this_year=0,

    # ====== 三.扣除額 (1+2+3) ======
    land_increment_tax=0,
    deed_tax=0,
    other_gift_burdens=0,

    # ====== 五.同一年內以前各次應納稅額及可扣抵稅額 ======
    previous_gift_tax_or_credit=0,

    # ====== 六.「新舊制差額調整」(主要用於106年) ======
    new_old_system_adjustment=0,

    # ====== 自由變數名稱列表 ======
    free_vars: list | None = None,
):
    """計算並最小化贈與稅。

    回傳 (最佳稅額, 課稅贈與淨額, 參數 dict, 差異 dict)。
    """

    # --------------------------------------------------
    # 0. 準備
    # --------------------------------------------------
    if free_vars is None:
        free_vars = []
    free_vars = set(free_vars)

    base_opt = Optimize()  # 用於最終最佳化

    # --------------------------------------------------
    # 1. 將所有輸入建立為 Z3 變數
    # --------------------------------------------------
    numeric_inputs = {
        "land_value": land_value,
        "ground_value": ground_value,
        "house_value": house_value,
        "others_value": others_value,
        "not_included_land": not_included_land,
        "not_included_house": not_included_house,
        "not_included_others": not_included_others,
        "remaining_exemption_98": remaining_exemption_98,
        "previous_gift_sum_in_this_year": previous_gift_sum_in_this_year,
        "land_increment_tax": land_increment_tax,
        "deed_tax": deed_tax,
        "other_gift_burdens": other_gift_burdens,
        "previous_gift_tax_or_credit": previous_gift_tax_or_credit,
        "new_old_system_adjustment": new_old_system_adjustment,
    }

    params: dict[str, tuple] = {}
    for name, val in numeric_inputs.items():
        z3v = Real(name)
        # 範圍約束：除 new_old_system_adjustment 可為負，其餘 >=0
        cons = [] if name == "new_old_system_adjustment" else [lambda v: v >= 0]
        params[name] = (z3v, val, cons)

    # --------------------------------------------------
    # 2. 加入固定/自由變數條件
    # --------------------------------------------------
    for pname, (z3v, pval, cons) in params.items():
        if pname in free_vars:
            for c in cons:
                base_opt.add(c(z3v))
        else:
            base_opt.add(z3v == pval)
            for c in cons:
                base_opt.add(c(z3v))

    # --------------------------------------------------
    # 3. 其他公式變數與核心公式
    # --------------------------------------------------
    this_gift_total = Real("this_gift_total")
    annual_gift_total = Real("annual_gift_total")
    net_taxable_income = Real("net_taxable_income")
    final_tax = Real("final_tax")
    final_tax_int = Int("final_tax_int")  # 取整稅額作為目標
    applied_rate = Real("applied_rate")
    progressive_diff = Real("progressive_diff")
    total_deductions = Real("total_deductions")

    # final_tax_int 為 floor(final_tax)，並且稅額不得為負
    base_opt.add(final_tax_int == ToInt(final_tax))
    base_opt.add(final_tax_int >= 0)

    V = lambda n: params[n][0]

    base_opt.add(this_gift_total == V("land_value") + V("ground_value") + V("house_value") + V("others_value"))
    base_opt.add(annual_gift_total == this_gift_total + V("previous_gift_sum_in_this_year"))
    base_opt.add(total_deductions == V("land_increment_tax") + V("deed_tax") + V("other_gift_burdens"))

    # --- 依期間公式 ---
    if period_choice == 1:
        base_opt.add(net_taxable_income == this_gift_total - V("remaining_exemption_98") - total_deductions)
        base_opt.add(applied_rate == 0.10, progressive_diff == 0)
        base_opt.add(final_tax == net_taxable_income * applied_rate - V("previous_gift_tax_or_credit"))

    elif period_choice == 2:
        exemption = 2_200_000
        base_opt.add(net_taxable_income == annual_gift_total - exemption - total_deductions)
        c1 = net_taxable_income <= 25_000_000
        c2 = And(net_taxable_income > 25_000_000, net_taxable_income <= 50_000_000)
        c3 = net_taxable_income > 50_000_000
        base_opt.add(Or(c1, c2, c3))
        base_opt.add(Implies(c1, And(applied_rate == 0.10, progressive_diff == 0)))
        base_opt.add(Implies(c2, And(applied_rate == 0.15, progressive_diff == 1_250_000)))
        base_opt.add(Implies(c3, And(applied_rate == 0.20, progressive_diff == 3_750_000)))
        base_opt.add(final_tax == net_taxable_income * applied_rate - progressive_diff - V("previous_gift_tax_or_credit") - V("new_old_system_adjustment"))

    elif period_choice == 3:
        exemption = 2_440_000
        base_opt.add(net_taxable_income == annual_gift_total - exemption - total_deductions)
        c1 = net_taxable_income <= 25_000_000
        c2 = And(net_taxable_income > 25_000_000, net_taxable_income <= 50_000_000)
        c3 = net_taxable_income > 50_000_000
        base_opt.add(Or(c1, c2, c3))
        base_opt.add(Implies(c1, And(applied_rate == 0.10, progressive_diff == 0)))
        base_opt.add(Implies(c2, And(applied_rate == 0.15, progressive_diff == 1_250_000)))
        base_opt.add(Implies(c3, And(applied_rate == 0.20, progressive_diff == 3_750_000)))
        base_opt.add(final_tax == net_taxable_income * applied_rate - progressive_diff - V("previous_gift_tax_or_credit"))

    elif period_choice == 4:
        exemption = 2_440_000
        base_opt.add(net_taxable_income == annual_gift_total - exemption - total_deductions)
        c1 = net_taxable_income <= 28_110_000
        c2 = And(net_taxable_income > 28_110_000, net_taxable_income <= 56_210_000)
        c3 = net_taxable_income > 56_210_000
        base_opt.add(Or(c1, c2, c3))
        base_opt.add(Implies(c1, And(applied_rate == 0.10, progressive_diff == 0)))
        base_opt.add(Implies(c2, And(applied_rate == 0.15, progressive_diff == 1_405_500)))
        base_opt.add(Implies(c3, And(applied_rate == 0.20, progressive_diff == 4_216_000)))
        base_opt.add(final_tax == net_taxable_income * applied_rate - progressive_diff - V("previous_gift_tax_or_credit"))

    else:
        raise ValueError("Unsupported period_choice")

    # --------------------------------------------------
    # 4. 計算 fixed 基準稅額（全部固定）供對照 計算 fixed 基準稅額（全部固定）供對照
    # --------------------------------------------------
    fixed_tax = None
    if free_vars:
        base_opt.push()
        for pname in free_vars:
            base_opt.add(params[pname][0] == params[pname][1])
        if base_opt.check() == sat:
            fixed_tax = base_opt.model().evaluate(ToInt(final_tax)).as_long()
        base_opt.pop()

    # --------------------------------------------------
    # 5. 設定最佳化目標 & 求解
    # --------------------------------------------------
    base_opt.minimize(final_tax_int)
    status = base_opt.check()
    if status != sat:
        return None, None, {}, {}

    mdl = base_opt.model()
    best_tax = mdl.evaluate(final_tax_int).as_long()
    taxable_net = mdl.evaluate(ToInt(net_taxable_income)).as_long()

    # --------------------------------------------------
    # 6. 彙整差異與參數
    # --------------------------------------------------
    final_params, differences = {}, {}
    for pname, (z3v, pval, _) in params.items():
        try:
            opt_val = mdl.eval(z3v).as_long()
        except Exception:
            opt_val = None
        final_params[pname] = {"value": opt_val, "type": "free" if pname in free_vars else "fixed"}
        if opt_val is not None and opt_val != pval:
            differences[pname] = {"original": pval, "optimized": opt_val, "difference": opt_val - pval}

    # --------------------------------------------------
    # 7. 印出摘要資訊
    # --------------------------------------------------
    print(f"固定參數初始稅額: {fixed_tax}")
    print(f"最佳化後稅額    : {best_tax}")
    if differences:
        print("變動參數:")
        for k, v in differences.items():
            print(f"  {k}: {v['original']} -> {v['optimized']} (Δ {v['difference']})")

    return best_tax, taxable_net, final_params, differences


# ------------------------ 簡單示例 ------------------------
if __name__ == "__main__":
    best_tax, taxable, fpar, diff = gift_tax_calculator(
        period_choice=4,
        land_value=60_000_000,
        ground_value=560_000,
        house_value=5_000_000,
        others_value=4_000_000,
        not_included_land=0,
        not_included_house=0,
        not_included_others=0,
        remaining_exemption_98=0,
        previous_gift_sum_in_this_year=3_000_000,
        land_increment_tax=890_000,
        deed_tax=1_230_000,
        other_gift_burdens=4_444_440,
        previous_gift_tax_or_credit=1_000_000,
        new_old_system_adjustment=0,
        free_vars=["land_value", "others_value"],
    )

    print("============================")
    print("最佳稅額:", best_tax)
    print("課稅淨額:", taxable)
    print("最終參數:", fpar)
    print("差異:", diff)


固定參數初始稅額: 7495112
最佳化後稅額    : 0
變動參數:
  land_value: 60000000 -> 10444440 (Δ -49555560)
  others_value: 4000000 -> 0 (Δ -4000000)
最佳稅額: 0
課稅淨額: 10000000
最終參數: {'land_value': {'value': 10444440, 'type': 'free'}, 'ground_value': {'value': 560000, 'type': 'fixed'}, 'house_value': {'value': 5000000, 'type': 'fixed'}, 'others_value': {'value': 0, 'type': 'free'}, 'not_included_land': {'value': 0, 'type': 'fixed'}, 'not_included_house': {'value': 0, 'type': 'fixed'}, 'not_included_others': {'value': 0, 'type': 'fixed'}, 'remaining_exemption_98': {'value': 0, 'type': 'fixed'}, 'previous_gift_sum_in_this_year': {'value': 3000000, 'type': 'fixed'}, 'land_increment_tax': {'value': 890000, 'type': 'fixed'}, 'deed_tax': {'value': 1230000, 'type': 'fixed'}, 'other_gift_burdens': {'value': 4444440, 'type': 'fixed'}, 'previous_gift_tax_or_credit': {'value': 1000000, 'type': 'fixed'}, 'new_old_system_adjustment': {'value': 0, 'type': 'fixed'}}
差異: {'land_value': {'original': 60000000, 'optimized': 1044